# Scatter points of the starting time in the progenitor parameter space

This notebook generates the scatter points of $t_{\text{start}}$ over progenitor parameter space, for a given ringdown model and $(\ell,|m|)$ harmonic.

**Imports & Settings**

In [1]:
# Standard library
import os

# Scientific / numeric
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

# Plotting
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns

# Plot config
rcParams['text.usetex'] = True
rcParams['font.size'] = 22


**Load mismatch results and NR simulations information**

In [3]:
"""
Load mismatch data distributions. These contain, for the London, Cheung and TEOBPM models, the (\ell,m) mismatch values for different starting times, where (\ell,m)={(2,2),(2,1),(3,3),(3,2),(4,4)}.
"""
!wget https://github.com/francesco-crescimbeni/Interpolating-function-of-ringdown-starting-time/raw/main/avg_mismatches_all_times.npz


"""
Load SXS catalog information (https://data.black-holes.org/simulations/index.html). To each simulation, we associate to it the triplet $(\eta,\chi_{+},\chi_{-})$.
"""
!wget https://github.com/francesco-crescimbeni/Interpolating-function-of-ringdown-starting-time/raw/main/SXS_BBH_nonprec_nonecc_all.txt

<>:2: SyntaxWarning: invalid escape sequence '\e'
<>:8: SyntaxWarning: invalid escape sequence '\e'
<>:2: SyntaxWarning: invalid escape sequence '\e'
<>:8: SyntaxWarning: invalid escape sequence '\e'
/tmp/ipython-input-495155260.py:2: SyntaxWarning: invalid escape sequence '\e'
  Load mismatch data distributions. These contain, for the London, Cheung and TEOBPM models, the (\ell,m) mismatch values for different starting times, where (\ell,m)={(2,2),(2,1),(3,3),(3,2),(4,4)}.
/tmp/ipython-input-495155260.py:8: SyntaxWarning: invalid escape sequence '\e'
  Load SXS catalog information (https://data.black-holes.org/simulations/index.html). To each simulation, we associate to it the triplet $(\eta,\chi_{+},\chi_{-})$.


--2025-11-01 20:00:49--  https://github.com/francesco-crescimbeni/Interpolating-function-of-ringdown-starting-time/raw/main/avg_mismatches_all_times.npz
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/francesco-crescimbeni/Interpolating-function-of-ringdown-starting-time/main/avg_mismatches_all_times.npz [following]
--2025-11-01 20:00:50--  https://raw.githubusercontent.com/francesco-crescimbeni/Interpolating-function-of-ringdown-starting-time/main/avg_mismatches_all_times.npz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 583752 (570K) [application/octet-stream]
Saving to: ‘avg_mismatches_al

In [ ]:
# List of avaliable models and modes
available_models = ["KerrBinary_London", "KerrBinary_Cheung", "TEOBPM"]
available_modes = ['22', '21', '33', '32', '44']

# Ask user for a valid model
print("Please choose a model:")
for model in available_models:
    print(f"- {model}")

model = input("Enter model name: ")

while model not in available_models:
    print("Invalid model. Please choose from the list.")
    model = input("Enter model name: ")

# Ask user for a valid mode
print("\nPlease choose a mode:")
for mode in available_modes:
    print(f"- {mode}")

mode = input("Enter mode: ")

while mode not in available_modes:
    print("Invalid mode. Please choose from the list.")
    mode = input("Enter mode: ")

# Final confirmation
print(f"\n✅ You selected model = '{model}' and mode = '{mode}'")

# Set parameters and load data
npz_file = 'avg_mismatches_all_times.npz'

# Load mismatch data
data = np.load(npz_file, allow_pickle=True)
avg_mismatches = data['avg_mismatches'].item()

# Load SXS catalog
catalog_file = 'SXS_BBH_nonprec_nonecc_all.txt'
thresholds = [0.035]
apply_shift = False  # change to True to apply t_shift logic

Please choose a model:
- KerrBinary_London
- KerrBinary_Cheung
- TEOBPM

Please choose a mode:
- 22
- 21
- 33
- 32
- 44

✅ You selected model = 'TEOBPM' and mode = '22'


## Load Data

In [ ]:
data = np.load(npz_file, allow_pickle=True)
avg_mismatches = data['avg_mismatches'].item()

id_info = {}
with open(catalog_file, "r") as f:
    for line in f:
        line=line.strip()
        if not line or line.startswith('#'): continue
        p=line.split()
        if len(p)<7: continue
        try:
            id_info[p[0]] = {"eta":float(p[2]), "chip":float(p[5]), "chim":float(p[6])}
        except: pass

print(f"Loaded {len(id_info)} simulations")

Loaded 336 simulations


## Define Plot Colors

In [ ]:
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16)/255.0 for i in (0, 2, 4))

def lighten_color(rgb_color, factor):
    return tuple(c + (1 - c) * factor for c in rgb_color)

cmap = LinearSegmentedColormap.from_list(
    "enhanced", [
        hex_to_rgb("#4477AA"),
        lighten_color(hex_to_rgb("#4477AA"), 0.4),
        (1,1,1),
        lighten_color(hex_to_rgb("#EE6677"), 0.4),
        hex_to_rgb("#EE6677")
    ], 256
)

## Main Plot Function

In [ ]:
def compute_t_start(sim_id, mode, threshold, start_times):
    ts, mm = [], []
    for t in start_times:
        v = avg_mismatches.get(model,{}).get(mode,{}).get(t,{}).get(sim_id,np.nan)
        if not np.isnan(v) and v>0:
            ts.append(t); mm.append(v)
    if len(ts)<2: return None
    ts, mm = np.array(ts), np.array(mm)
    for t, m in zip(ts, mm):
        if m <= threshold: return t
    return None

## Execute Scatter Plotting

In [ ]:
# LaTeX labels for axes
label_map = {"eta": r"\eta", "chip": r"\chi_+", "chim": r"\chi_-"}

for mode in available_modes:
    start_times = sorted(float(t) for t in avg_mismatches[models[0]][mode].keys())
    l_val, m_val = mode[0], mode[1]

    for threshold in thresholds:
        fig, axs = plt.subplots(3, len(models), figsize=(20 * len(models), 20))

        for col, model in enumerate(models):
            records = []
            for sim, prm in id_info.items():
                t_s = compute_t_start(sim, mode, threshold, start_times)
                records.append({**prm, "t": t_s})

            df = pd.DataFrame(records)

            for r, (x, y) in enumerate([("eta","chip"),("eta","chim"),("chip","chim")]):
                ax = axs[r][col]
                sc = ax.scatter(df[x], df[y], c=df["t"], cmap=cmap, vmin=0, vmax=30, s=60)

                ax.set_xlabel(f"${label_map[x]}$")
                ax.set_ylabel(f"${label_map[y]}$")

                if r == 0:
                    ax.set_title(rf"{model}, $({l_val},{m_val})$")

        fig.colorbar(sc, ax=axs.ravel().tolist(), shrink=0.9)

        # ---- SAVE TO PDF ----
        outname = f"scatter_lm{mode}_thresh_{threshold:.3f}.pdf"

        plt.savefig(outname)
        plt.close()

        print(f"✅ Saved: {outname}")


✅ Saved: scatter_lm22_thresh_0.035.pdf
✅ Saved: scatter_lm21_thresh_0.035.pdf
✅ Saved: scatter_lm33_thresh_0.035.pdf
✅ Saved: scatter_lm32_thresh_0.035.pdf
✅ Saved: scatter_lm44_thresh_0.035.pdf
